# Silver Layer Tables - Robot Events
![](img/silver_diagram.png)

This script defines **three Silver layer tables** in Databricks Delta Live Tables, built on top of the Bronze layer (`bronze_robot_events`).  
They apply **data quality checks**, **cleansing**, and **schema flattening** for downstream analytics.

---

## 1. `silver_command_events`
**Purpose:** Cleaned and filtered robot command execution events.  
- **Quality rules**:
  - Drop rows with invalid `duration_ms` (`> 0`)
  - Ensure `battery_level` is between `0` and `100`
- **Transformations**:
  - Filter by `event_type = command_execution`
  - Flatten nested fields (command details, location)
  - Ensure valid timestamp
- **Output columns**:
  - `robot_id`, `timestamp`, `x`, `y`, `command_name`, `duration_ms`, `success`, `battery_level`, `obstacle_detected`

---

## 2. `silver_sensor_data`
**Purpose:** Cleaned robot sensor readings.  
- **Quality rules**:
  - Ensure `battery_level` between `0` and `100`
  - Ensure `battery_temp_c` between `0` and `50`
- **Transformations**:
  - Extract battery and LiDAR sensor data
  - Ensure valid timestamp
- **Output columns**:
  - `robot_id`, `timestamp`, `battery_level`, `battery_temp_c`, `obstacle_detected`, `scan_points`

---

## 3. `silver_combined_events`
**Purpose:** Unified table combining command, sensor, and location data.  
- **Quality rules**:
  - Drop rows with invalid `duration_ms` (`> 0`)
  - Ensure `battery_level` between `0` and `100`
- **Transformations**:
  - Filter by `event_type = command_execution`
  - Flatten and merge command + sensor + location fields
  - Ensure valid timestamp
- **Output columns**:
  - `robot_id`, `timestamp`, `x`, `y`, `command_name`, `duration_ms`, `success`, `battery_level`, `obstacle_detected`

---

> ⚡ **Tip:** Update `CATALOG`, `BRONZE`, and `SILVER` variables to match your environment before running.


In [0]:
import dlt
from pyspark.sql.functions import col, when, to_timestamp

CATALOG = "haley_b_demo"
BRONZE = "bronze"
SILVER = "silver"

@dlt.table(name=f"{CATALOG}.{SILVER}.silver_command_events", comment="Cleaned and filtered command events")
@dlt.expect_or_drop("valid_duration", "duration_ms > 0")
@dlt.expect("battery_in_range", "battery_level BETWEEN 0 AND 100")
def silver_command_events():
    df = dlt.read(f"{CATALOG}.{BRONZE}.bronze_robot_events")

    # Filter
    df = df.filter(col("event.event_type") == "command_execution")

    # Cleansing and flatten
    df = df.withColumn("duration_ms", when(col("event.command.duration_ms") > 0, col("event.command.duration_ms")).otherwise(None))
    df = df.withColumn("battery_level", when((col("sensors.battery.level_percent") >= 0) & (col("sensors.battery.level_percent") <= 100), col("sensors.battery.level_percent")).otherwise(None))

    # Timestamp filtering
    df = df.filter(col("timestamp").isNotNull())
    df = df.withColumn("timestamp", to_timestamp(col("timestamp")))

    return df.select(
        "robot_id",
        "timestamp",
        "location.x",
        "location.y",
        col("event.command.name").alias("command_name"),
        "duration_ms",
        col("event.command.success").alias("success"),
        "battery_level",
        col("sensors.lidar.obstacle_detected").alias("obstacle_detected")
    )

In [0]:
CATALOG = "haley_b_demo"
BRONZE = "bronze"
SILVER = "silver"

@dlt.table(name=f"{CATALOG}.{SILVER}.silver_sensor_data", comment="Cleaned robot sensor data")
@dlt.expect("battery_level_in_range", "battery_level BETWEEN 0 AND 100")
def silver_sensor_data():
    df = dlt.read(f"{CATALOG}.{BRONZE}.bronze_robot_events")

    df = df.withColumn("battery_level", when((col("sensors.battery.level_percent") >= 0) & (col("sensors.battery.level_percent") <= 100), col("sensors.battery.level_percent")).otherwise(None))
    df = df.withColumn("battery_temp_c", when((col("sensors.battery.temperature_c") >= 0) & (col("sensors.battery.temperature_c") <= 50), col("sensors.battery.temperature_c")).otherwise(None))

    df = df.filter(col("sensors.lidar.obstacle_detected").isNotNull())
    df = df.filter(col("timestamp").isNotNull())
    df = df.withColumn("timestamp", to_timestamp(col("timestamp")))

    return df.select(
        "robot_id",
        "timestamp",
        "battery_level",
        "battery_temp_c",
        col("sensors.lidar.obstacle_detected").alias("obstacle_detected"),
        col("sensors.lidar.scan_points").alias("scan_points")
    )

In [0]:
CATALOG = "haley_b_demo"
BRONZE = "bronze"
SILVER = "silver"

@dlt.table(name=f"{CATALOG}.{SILVER}.silver_combined_events", comment="Combined robot command + sensor + location data")
@dlt.expect_or_drop("valid_duration", "duration_ms > 0")
@dlt.expect("battery_level_in_range", "battery_level BETWEEN 0 AND 100")
def silver_combined_events():
    df = dlt.read(f"{CATALOG}.{BRONZE}.bronze_robot_events")

    df = df.filter(col("event.event_type") == "command_execution")

    df = df.withColumn("duration_ms", when(col("event.command.duration_ms") > 0, col("event.command.duration_ms")).otherwise(None))
    df = df.withColumn("battery_level", when((col("sensors.battery.level_percent") >= 0) & (col("sensors.battery.level_percent") <= 100), col("sensors.battery.level_percent")).otherwise(None))
    df = df.filter(col("timestamp").isNotNull())
    df = df.withColumn("timestamp", to_timestamp(col("timestamp")))

    return df.select(
        "robot_id",
        "timestamp",
        col("location.x").alias("x"),
        col("location.y").alias("y"),
        col("event.command.name").alias("command_name"),
        "duration_ms",
        col("event.command.success").alias("success"),
        "battery_level",
        col("sensors.lidar.obstacle_detected").alias("obstacle_detected")
    )